<font size="10" color="black">Exercício Aula 6 - Avaliação de Políticas Públicas</font>

Eduardo Chaves Ferreira

## Importação de bibliotecas usadas nos exemplos

In [ ]:


import numpy as np
import matplotlib.pyplot as plt
import math
import random
import pandas as pd
import scipy.stats as stat
from collections import Counter
from scipy.stats import t
from scipy.stats import norm
from sklearn.utils import shuffle

import os

path = os.environ['PATH']

if path.startswith('C'):
    IN_KAGGLE = False
else:
    IN_KAGGLE = True

## Mantendo a reprodutibilidade dos resultados

Antes da geração de números aleatórios é importante inicializar o gerador de números para que os resultados sejam os mesmos

In [ ]:
# Para uso com funções da biblioteca standard (ex random.randint)
random.seed(1)
# Para uso com funções da biblioteca numpy (ex np.random.randint)
np.random.seed(1)

# Quando for passada como parâmetro a seed
random_state = 1

## Funções gerais usadas nos exemplos

In [ ]:
# Calcula número de desvios na distribuição t de student para determinada confianca


def calcula_Z_tstudent(confianca, tamanho_amostra):
    mean = 0
    std = 1
    rv = t(df=(tamanho_amostra-1))
    return rv.interval(confianca)[1]

In [ ]:
# Calcula número de desvios na distribuição normal para determinada confianca


def calcula_Z_normal(confianca):
    mean = 0
    std = 1
    rv = norm(loc=mean, scale=std)
    return rv.interval(confianca)[1]

In [ ]:
# Calcula número de desvios na distribuição normal para determinada confianca e múltiplas proporções

def calcula_Z_Multiplas_Proporcoes(confianca, numero_proporcoes):
    area = math.pow(  confianca, 1/(numero_proporcoes-1) )
    return calcula_Z_normal(area)



In [ ]:
# Recupera amostra da população

def recupera_amostra(populacao, tamanho_amostra):
    tamanho_amostra = int(tamanho_amostra)
    if type(populacao) is pd.DataFrame:
        return populacao.sample(tamanho_amostra)
    else:
        #return populacao[np.random.randint(0, len(populacao), tamanho_amostra)]
        return [populacao[i] for i in np.random.randint(0, len(populacao), tamanho_amostra)]

In [ ]:
# Função que calcula intervalo de confiança para média com base em uma amostra e a confiança desejada

def calcula_intervalo_media(amostra, confianca, tamanho_populacao):
    tamanho_amostra = len(amostra)
    
    #1-Calcule a média da amostra ex. media_amostra = np.mean(amostra)
    media_amostra = np.mean(amostra)

    #2-Calcule o desvio da amostra ex. desvio_amostra = np.std(amostra)
    desvio_amostra = np.std(amostra)

    #3-Calcule quantos desvios precisará para seu grau de confiânça ex. numero_desvios = calcula_numero_desvios_tstudent_para_confianca(confiança, tamanho_amostra)
    numero_desvios = calcula_Z_tstudent(confianca, tamanho_amostra)

    #4-Calcule o desvio das amostras ex. desvio_amostras = desvio_amostra/np.sqrt(tamanho_amostra)
    desvio_amostras = desvio_amostra/np.sqrt(tamanho_amostra)

    #5-Calcule a margem de erro ex. margem_erro = numero_desvios*desvio_amostras
    margem_erro = numero_desvios*desvio_amostras
    
    if tamanho_amostra>0.05*tamanho_populacao:
        margem_erro = margem_erro * (np.sqrt(tamanho_populacao-tamanho_amostra)/np.sqrt(tamanho_populacao-1))

    #6-Calcule o intervalo ex. inferior = media_amostra-margem_erro, superior = media_amostra+margem_erro
    inferior = media_amostra-margem_erro
    superior = media_amostra+margem_erro

    return inferior,superior

In [ ]:
# Função que calcula intervalo de confiança para proporção de determinado valor com base em uma amostra e a confiança desejada
# O parâmetro valor representa a categoria na amostra para a qual se deseja o intervalo de confiança da proporção, por exemplo o nome de um candidato

def calcula_intervalo_proporcao(amostra, confianca, valor, tamanho_populacao):
    tamanho_amostra = len(amostra)
    
    proporcao_valor = Counter(amostra)[valor]/tamanho_amostra
    
    numero_desvios = calcula_Z_tstudent(confianca, tamanho_amostra)

    margem_erro = numero_desvios*np.sqrt(proporcao_valor*(1-proporcao_valor))/np.sqrt(tamanho_amostra)
    
    if tamanho_amostra>0.05*tamanho_populacao:
        margem_erro = margem_erro * (np.sqrt(tamanho_populacao-tamanho_amostra)/np.sqrt(tamanho_populacao-1))

    inferior = proporcao_valor-margem_erro
    superior = proporcao_valor+margem_erro

    return inferior,superior

In [ ]:
# Função para cálculo do número de amostras para médias e somas

def Tamanho_Amostra_Valor_Pontual(Tamanho_Populacao, Grau_Confianca, Variancia, Margem_Erro):
    Numero_Desvios = calcula_Z_normal(Grau_Confianca)
    return math.ceil( 
            (Tamanho_Populacao * math.pow(Numero_Desvios,2) * Variancia) / \
            ( (Tamanho_Populacao-1)*math.pow(Margem_Erro,2) + math.pow(Numero_Desvios,2)*Variancia )
            )

In [ ]:
# Função para cálculo do número de amostras para proporções simples

def Tamanho_Amostra_Proporcao_Simples(Tamanho_Populacao, Grau_Confianca, Proporcao, Margem_Erro):
    Numero_Desvios = calcula_Z_normal(Grau_Confianca)
    return math.ceil( 
            (Tamanho_Populacao * math.pow(Numero_Desvios,2) * Proporcao * (1-Proporcao)) / \
            ( (Tamanho_Populacao-1)*math.pow(Margem_Erro,2) + math.pow(Numero_Desvios,2)*Proporcao*(1-Proporcao) )
            )

In [ ]:
# Função para cálculo do tamanho da amostra para proporções múltiplas

def Tamanho_Amostra_Proporcao_Multipla(Tamanho_Populacao, Confianca, Numero_Proporcoes, Margem_Erro):
    Confianca_ = calcula_Z_Multiplas_Proporcoes(Confianca, Numero_Proporcoes)
    return math.ceil( 
            (Tamanho_Populacao * math.pow(Confianca_,2) * 0.25 ) / \
            ( (Tamanho_Populacao-1)*math.pow(Margem_Erro,2) + math.pow(Confianca_,2)*0.25 )
            )

In [ ]:
def grafico(planilha, coluna):
    planilha[coluna].value_counts().plot(kind='bar')
    plt.xlabel(coluna)
    plt.ylabel('Quantidade')

    plt.title(coluna)
    plt.grid(True)
    plt.show()
    
    print(planilha[coluna].value_counts()/len(planilha))

In [ ]:
def histograma(planilha, coluna):
    n, bins, patches = plt.hist(planilha[coluna],  facecolor='g', alpha=0.75, bins=50)
    plt.xlabel(coluna)
    plt.ylabel('Quantidade')

    plt.title(coluna)
    plt.grid(True)
    plt.show()
    
    print('Média {}, desvio padrão {}'.format(planilha[coluna].mean(),planilha[coluna].std()))


In [ ]:
def aplicaPolitica(PopulacaoControle,PopulacaoTeste ):
    PopulacaoControlePosPolitica = PopulacaoControle.copy()
    PopulacaoControlePosPoliticaMediaSalario = PopulacaoControlePosPolitica.Salario.mean()
    PopulacaoControlePosPoliticaMediaSalario = PopulacaoControlePosPoliticaMediaSalario * 1.1
    PopulacaoControlePosPolitica.Salario = PopulacaoControlePosPolitica.Salario * norm(loc=PopulacaoControlePosPoliticaMediaSalario, scale=0.1*PopulacaoControlePosPoliticaMediaSalario).rvs(size=TamanhoPopulacaoControle)

    PopulacaoTestePosPolitica = PopulacaoTeste.copy()
    PopulacaoTestePosPoliticaMediaSalario = PopulacaoTestePosPolitica.Salario.mean()
    PopulacaoTestePosPoliticaMediaSalario = PopulacaoTestePosPoliticaMediaSalario * 1.3
    PopulacaoTestePosPolitica.Salario = PopulacaoTestePosPolitica.Salario * norm(loc=PopulacaoTestePosPoliticaMediaSalario, scale=0.1*PopulacaoTestePosPoliticaMediaSalario).rvs(size=TamanhoPopulacaoTeste)

    return PopulacaoControlePosPolitica, PopulacaoTestePosPolitica

<font size="6" color="red">Análise de dados</font>

# Utilize uma base já carregada

No modo de edição do notebook, canto superior direito, selecione "Add Data"

Procure a base que deseja, neste exemplo "VotacaoSimuladaEleicaoPresidencialBrasil2018"

# Leia os dados

In [ ]:
populacao = pd.read_excel('../input/votacaosimuladaeleicaopresidencialbrasil2018/populacao.xlsx', index_col=0) 
   

In [ ]:

populacao.head()

In [ ]:
#len(populacao)

# Visualize os dados carregados

Há funções para visualização de dados numéricos (histograma) e categóricos (grafico). 

Ex: histograma(populacao,'Salario'), grafico(populacao,'Regiao')

In [ ]:
# Salario
histograma(populacao,'Salario')

In [ ]:
# Idade
histograma(populacao,'Idade')

In [ ]:
# Regiao
grafico(populacao,'Regiao')

In [ ]:
# Voto
grafico(populacao,'Voto')

In [ ]:
# Sexo
grafico(populacao,'Sexo')

# O que queremos?

O primeiro passo do trabalho é saber onde quer chegar

Na análise de política pública normalmente queremos descobrir se uma política teve impacto e quantificá-lo (na análise quantitativa)

Para isso precisamos seguir um conjunto de passos:

1- Descobrir o tamanho mínimo da amostra para fazer inferências sobre a população

2- Criar essa amostra

3- Calcular o parâmetro que queremos na amostra e o intervalo de canfiança para a população

4- Fazer os passos acima para os dois grupos em análise (teste e controle)

5- Comparar os resultados dos grupos para verificar se há diferenças significativas, ou seja, se houve impacto decorrente da política (teste de hipótese)

# Exercício

Vamos começar com um exercício simples: vamos estimar a proporção de sexo (M e F), média salarial e proporção de votos (B H C I) para a população em análise

# Primeiro passo

Você deve classificar as informações que deseja como:
1- Valor pontual - Exemplo: salário - Variável numérica ( como o próprio nome indica, diz respeito a algo aferível numericamente como o salário de uma pessoa)
2- Proporção simples ( somente pode haver duas resposta. Como o sexo da pessoa. Ou é feminino ou é masculino) - Exemplo: sexo - Variável categórica
3- Proporção múltipla - Exemplo: número de regiões do país - Variável categórica


# Segundo passo
Para determinar o tamanho da amostra você precisa definir a confiânça (95%) e margem de erro para salário (R$ 100), Sexo (5%) e Votos (5%)

In [ ]:
Margem_Erro_Salario = 100
Margem_Erro_Proporcao_Sexo = 0.05
Margem_Erro_Proporcao_Votos = 0.05
Grau_Confianca = 0.95


# Amostra piloto

Para proporção simples (Sexo) e Salário precisamos de uma amostra piloto para estimar a variância populacional

Vamos extrair uma amostra de 30 elementos e calcular a variância para Salário e a proporção para Sexo masculino

In [ ]:

Tamanho_Amostra = 30
#resgata a fórmula disponibilizada acima
amostra = recupera_amostra(populacao, Tamanho_Amostra)

Variancia_Salario_Estimada = amostra.Salario.var()
print(Variancia_Salario_Estimada)

Proporcao_Sexo_M = sum(amostra.Sexo=='M')/Tamanho_Amostra
print(Proporcao_Sexo_M)


In [ ]:
amostra.head()


# Descubra o tamanho aproximado da população

Esta informação não seria calculada na prática, tendo em vista que você não terá acesso a todos os dados da população.

Neste exercício, como você tem a planilha populacao, pode calcular seu tamanho

In [ ]:
#Tamanho_Populacao = 
#tamanho_amostra = len(amostra)
Tamanho_Populacao=len(populacao)
print(Tamanho_Populacao)

# Cálculo tamanho amostra para Salário

In [ ]:


#Tamanho_Amostra_Salario = 
Tamanho_Amostra_Salario = Tamanho_Amostra_Valor_Pontual (Tamanho_Populacao, Grau_Confianca, Variancia_Salario_Estimada, Margem_Erro_Salario)

Tamanho_Amostra_Salario

# Cálculo tamanho amostra para Sexo masculino

In [ ]:


#Tamanho_Amostra_Sexo = 

#Tamanho_Amostra_Sexo
Tamanho_Amostra_Sexo = Tamanho_Amostra_Proporcao_Simples(Tamanho_Populacao, Grau_Confianca, Proporcao_Sexo_M, Margem_Erro_Proporcao_Sexo)
Tamanho_Amostra_Sexo

# Cálculo tamanho amostra para Votação

In [ ]:


#Tamanho_Amostra_Votacao = 

#Tamanho_Amostra_Votacao
Tamanho_Amostra_Votacao = Tamanho_Amostra_Proporcao_Multipla(Tamanho_Populacao, Grau_Confianca, 4, Margem_Erro_Proporcao_Votos)
Tamanho_Amostra_Votacao

# Escolha o maior tamanho entre todos os valores calculados e faça a amostra

Na prática este passo seria realizado através de um levantamento de dados (ex. pesquisa)

No caso, a pesquisa já foi realizada e o resultado está num Excel disponibilizado no Moodle (amostra.xlsx)

# Importe seu arquivo em Excel ou CSV

No modo de edição do notebook, canto superior direito, selecione "Add Data"

Faça upload dos dados

In [ ]:
#amostra = pd.read_excel('../input/amostra.xlsx', index_col=0) 
#amostra.head()

# Calcule o intervalo de confiânça para a média de salários na população

Compare o intervalo com a média real da população

Compare a margem de erro calculada com a margem de erro inicialmente proposta

In [ ]:
#inferior,superior = 
#print('Intervalo da média {} {}'.format(inferior,superior))
#print('Margem de erro {}'.format((superior-inferior)/2))

# Calcule o intervalo de confiânça para a proporção de sexo masculino na população

Compare o intervalo com a proporção real da população

Compare a margem de erro calculada com a margem de erro inicialmente proposta

In [ ]:
#inferior,superior = 
#print('Intervalo de confiança M {} {}'.format(inferior,superior))

#print('Margem de erro {}'.format((superior-inferior)/2))

# Calcule o intervalo de confiânça para a proporção de votos do candidato B na população

Compare o intervalo com a proporção real da população

Compare a margem de erro calculada com a margem de erro inicialmente proposta

In [ ]:
#inferior,superior = 
#print('Intervalo de confiança B {} {}'.format(inferior,superior))

#print('Margem de erro {}'.format((superior-inferior)/2))

# Estime a idade média da população e a proporção de habitantes da região sudeste

Calcule o tamanho da amostra necessário para ambas as variáveis

A amostra fornecida tem tamanho suficiente para realizar os cálculos pedidos?

<font size="6" color="red">Teste de Hipótese</font>

Até agora não testamos nenhuma hipótese, apenas estimamos parâmetros populacionais através de estatísticas amostrais

Vamos considerar agora que foi implementada uma política de educação de adultos na região nordeste, com o objetivo de melhorar a condição salarial dos participantes

O programa foi aplicado para trabalhadores de baixa renda (até R$ 900,00)

A abrangência foi de 30% do público potencial

In [ ]:
'''
PopulacaoPotencial = populacao.loc[(populacao.Regiao=='Nordeste') & (populacao.Salario<=900),:].copy()
TamanhoPopulacaoPotencial = len(PopulacaoPotencial)
TamanhoPopulacaoPotencial
'''

# Vamos criar os grupos de teste e controle

In [ ]:
'''
from sklearn.model_selection import train_test_split

PopulacaoControle, PopulacaoTeste = train_test_split(PopulacaoPotencial, test_size=0.3)
'''

In [ ]:
'''
TamanhoPopulacaoTeste = len(PopulacaoTeste)
TamanhoPopulacaoTeste
'''

In [ ]:
'''
TamanhoPopulacaoControle = len(PopulacaoControle)
TamanhoPopulacaoControle
'''

# Cálculos antes da política

Calcule o tamanho mínimo de amostra para estimar a média de salário da população de teste e controle

Faça a amostra e estime o intervalo de confiânça para a média de salários dos dois grupos antes da aplicação da política

# Após a implantação da política

In [ ]:
#PopulacaoControlePosPolitica, PopulacaoTestePosPolitica = aplicaPolitica(PopulacaoControle,PopulacaoTeste )

# Cálculos após a política

Faça a amostra e estime a média de salário dos dois grupos após a implantação da política

# Torne o notebook público e mande o link para o professor